# Formating experimental results

In [1]:
%matplotlib inline
import os
import sys
import numpy as np
import pickle as pkl
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from models import MTC

In [3]:
datasets = ['aotm2011', '30music']
metrics = ['R-Precision', 'AUC', 'NDCG']
TOPs = [5, 10, 20, 30, 50, 100, 200, 300, 500, 1000]

## Playlist augmentation

In [4]:
dix = 1
task = 2
dataset_name = datasets[dix]
data_dir = 'data/%s/setting%d' % (dataset_name, task)

In [ ]:
algo_names = ['Logistic Regression', 'Popularity Rank', 'Multitask Classification', 'SAGH', 'CAGH']
algos = ['br2', 'pop', 'pla', 'sagh', 'cagh']

### Performance table

In [ ]:
#col_indices = pd.MultiIndex.from_product([algo_names, ['K=%d' % k for k in range(1,5)]])
rows = ['K=%d' % k for k in range(1,5)]
cols = pd.MultiIndex.from_product([algo_names, metrics])

In [ ]:
df = pd.DataFrame(index=rows, columns=cols)

In [ ]:
for j in range(len(algos)):
    algo_name = algo_names[j]
    algo = algos[j]
    for k in range(1,5):
        fperf = os.path.join(data_dir, 'perf-%s-%d.pkl' % (algo, k))
        if not os.path.exists(fperf):
            continue
        perf_dict = pkl.load(open(fperf, 'rb'))
        #print(perf_dict)
        for metric in metrics:
            df.loc['K=%d' % k, (algo_name, metric)] = perf_dict[dataset_name]['Test'][metric] * 100

In [ ]:
df

In [ ]:
tab_str = df.to_latex(float_format=lambda x: '$%.1f$' % x, na_rep='N/A', \
                      column_format='l|' + '*{%d}{c}|' % len(metrics) * len(algos), \
                      multicolumn=True, multicolumn_format='c', escape=False)

In [ ]:
print('\\begin{table*}[!h]')
print('\centering')
print('\\caption{Performance $\\times 10^2$ (%s)}' % dataset_name)
print('\\label{tab:perf_pla_%s}' % dataset_name)
print('\\resizebox{\\linewidth}{!}{')
print(tab_str)
print('}')
print('\\end{table*}')

### Hit Rate curve

In [ ]:
metric = 'Hit-Rate'

In [ ]:
nrows, ncols = 1, 4
colors = ['r', 'g', 'b', 'm', 'c']
fig = plt.figure(figsize=[20, 3])
for k in range(1,5):
    fperfs = [os.path.join(data_dir, 'perf-%s-%d.pkl' % (algo, k)) for algo in algos]
    perf_dicts = [pkl.load(open(fperf, 'rb')) if os.path.exists(fperf) else None for fperf in fperfs]
    for j in range(len(perf_dicts)):
        if perf_dicts[j] is None:
            continue
        x = TOPs
        y = [perf_dicts[j][dataset_name]['Test'][metric][k] for k in TOPs]
        ax = plt.subplot(nrows, ncols, k)
        ax.plot(x, y, ls='-', c=colors[j], label=algo_names[j])
        ax.set_xscale('log')
        ax.set_title('K=%d' % k)
    plt.legend(loc='best')
    plt.xlabel('Number of recommendations')
    if k == 1:
        plt.ylabel('Hit rate')
#plt.suptitle('Hit rates of playlist augmentation (%s)' % dataset_name, fontsize=12)
plt.savefig('%s-%d.svg' % (dataset_name, task))

## New song recommendation

In [15]:
algo_names = ['Logistic Regression', 'PopTest Rank', 'LogReg Popularity', 'Multitask Classification']
algos = ['br1', 'poptest', 'lrpop', 'nsr']

### Performance table

In [16]:
rows = datasets
cols = pd.MultiIndex.from_product([algo_names, metrics])

In [17]:
df = pd.DataFrame(index=rows, columns=cols)

In [18]:
for dataset_name in datasets:
    for j in range(len(algos)):
        algo_name = algo_names[j]
        algo = algos[j]
        fperf = os.path.join('data/%s/setting1/perf-%s.pkl' % (dataset_name, algo))
        if not os.path.exists(fperf):
            continue
        perf_dict = pkl.load(open(fperf, 'rb'))
        #print(perf_dict)
        for metric in metrics:
            df.loc[dataset_name, (algo_name, metric)] = perf_dict[dataset_name]['Test'][metric] * 100

In [19]:
df

Logistic Regression                   PopTest Rank                    \
                 R-Precision      AUC     NDCG  R-Precision      AUC     NDCG   
aotm2011            0.170332  60.0077  9.52836      1.75871  86.6361  17.8632   
30music             0.644975  63.7959  11.2373      2.72043  80.3406  15.4544   

         LogReg Popularity                 Multitask Classification           \
               R-Precision    AUC     NDCG              R-Precision      AUC   
aotm2011               NaN    NaN      NaN                      NaN      NaN   
30music            2.60939  73.62  17.3859                 0.318027  63.3299   

                   
             NDCG  
aotm2011      NaN  
30music   10.4452

In [ ]:
tab_str = df.to_latex(float_format=lambda x: '$%.1f$' % x, na_rep='N/A', \
                      column_format='l|' + '*{%d}{c}|' % len(metrics) * len(algos), \
                      multicolumn=True, multicolumn_format='c', escape=False)

In [ ]:
print('\\begin{table*}[!h]')
print('\centering')
print('\\caption{Performance $\\times 10^2$ (%s)}' % dataset_name)
print('\\label{tab:perf_pla_%s}' % dataset_name)
print('\\resizebox{\\linewidth}{!}{')
print(tab_str)
print('}')
print('\\end{table*}')

### Hit Rate curve

In [ ]:
metric = 'Hit-Rate'

In [ ]:
nrows, ncols = 1, 4
colors = ['r', 'g', 'b', 'm', 'c']
fig = plt.figure(figsize=[20, 3])
for k in range(1,5):
    fperfs = [os.path.join(data_dir, 'perf-%s-%d.pkl' % (algo, k)) for algo in algos]
    perf_dicts = [pkl.load(open(fperf, 'rb')) if os.path.exists(fperf) else None for fperf in fperfs]
    for j in range(len(perf_dicts)):
        if perf_dicts[j] is None:
            continue
        x = TOPs
        y = [perf_dicts[j][dataset_name]['Test'][metric][k] for k in TOPs]
        ax = plt.subplot(nrows, ncols, k)
        ax.plot(x, y, ls='-', c=colors[j], label=algo_names[j])
        ax.set_xscale('log')
        ax.set_title('K=%d' % k)
    plt.legend(loc='best')
    plt.xlabel('Number of recommendations')
    if k == 1:
        plt.ylabel('Hit rate')
#plt.suptitle('Hit rates of playlist augmentation (%s)' % dataset_name, fontsize=12)
plt.savefig('%s-%d.svg' % (dataset_name, task))